### Sinogram_Production_Script

This script is used to combine the individual 1D-projections to sinogram data for the 2DeteCT dataset.

#### Initial imports

In [1]:
# This file contains a function for combining individual 1D-projections to sinogram data for the 2DeteCT dataset.

import astra
import imageio
import warnings
from typing import Any

import glob # Used for browsing through the directories.
import os # Used for creating directories.
import shutil # Later used for copying files.
import time # Used for keeping processing time.
from tqdm.notebook import tqdm_notebook # Used for tracking progress.

import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Data directories.

base_dir_str_1 = "/export/scratch1/mbk/2DeteCT_dataset/"
base_dir_str_2 = "/export/scratch2/mbk/2DeteCT_dataset/"
base_dir_str_add = "/export/scratch2/mbk/2DeteCT_add/"

save_dir_str = "/export/scratch2/mbk/2DeteCT_tests/test1/" 

In [3]:
# Keep track of the processing time per folder.
t = time.time();
print('Sanity check the data', flush=True)

# Get a list of all scan folders.
scan_folders_1 = sorted(glob.glob(base_dir_str_1 + "*"))
scan_folders_2 = sorted(glob.glob(base_dir_str_2 + "*"))
scan_folders_add = sorted(glob.glob(base_dir_str_add + "*"))
scan_folders = scan_folders_1 + scan_folders_2 + scan_folders_add

# Reference values for the number of files of full scan sets of 10, 20, 30, 40 and 50 slices.
reference_values = [108045,216090,324135,432180,540225]
reference_slice_nrs = [10,20,30,40,50]

# Nomenclatura.
modes = [1,2,3]
slcs_name ="slice{:05}"
projs_name = "scan_{:06}.tif"
dark_name = 'di000000.tif'
flat_name = ['io000000.tif', 'io000001.tif']
projs_idx = np.arange(3601)
flaws = False

# Create a dictionary to bookkeep the feasible/"good" slices.
good_slcs = dict()

# Enter the scan folders and check for completeness and ensure there are no errors.
for folder in tqdm_notebook(scan_folders):
    
    #Define a breaking variable if flaws are noticed.
    flaws = False
    sc1_last_slc = 0
    
    # 1. sanity check: are there enough files in the directory.
    num_files = len((glob.glob(folder + "/*/*/*.*")))
    for i_nr in np.arange(len(reference_values)):
        if(num_files <= reference_values[i_nr]):
            sc1_last_slc = reference_slice_nrs[i_nr-1]
            break
        sc1_last_slc = reference_slice_nrs[i_nr]
    print("By sanity check 1:", folder," has", sc1_last_slc, " feasible slices.")
    
    
    # 2. sanity check: are all files complete and correctly named.
    # * check if there is a dark and flat field in every first slice of a ten slice chunk.
    # * check if there is a flat field in every tenth slice of a ten slice chunk.
    # * check if all 3601 projection files are in every slice of a ten slice chunk.
    
    # Create a temporary variable for the total number of slices in this scan_folder and the last feasible slice.
    nr_slices = len((glob.glob(folder + "/*/mode1"))) # Any mode suffices for this query.
    last_slc = 0
    
    # Iterate through all slices...
    for i_slc in np.arange(1,nr_slices+1):
        
        # ... and all modes to check for completeness.
        for i_mode in modes:
            current_path = folder + "/" + slcs_name.format(i_slc) + "/mode{}/".format(i_mode)

            if i_slc in [1,11,21,31,41]:
                # Check if first slice of a ten chunk contains a flat field.
                if(glob.glob(current_path + flat_name[0])):
                    # Update the last feasible tenth package in this scan_folder.
                    last_slc = i_slc
                else:
                    # Update the last feasible tenth package in this scan_folder.
                    last_slc = int(i_slc-i_slc%10)
                    flaws = True
                    break

                # Check if first slice contains a dark field.
                if(glob.glob(current_path + dark_name)):
                    # Update the last feasible tenth package in this scan_folder.
                    last_slc = i_slc
                else:
                    # Update the last feasible tenth package in this scan_folder.
                    last_slc = int(i_slc-i_slc%10)
                    flaws = True
                    break

                # Check if all slices contain 3601 projections.
                for idx in projs_idx:
                    if(glob.glob(current_path + projs_name.format(projs_idx[idx]))):
                        # Update the last feasible tenth package in this scan_folder.
                        last_slc = i_slc
                    else:
                        # Update the last feasible tenth package in this scan_folder.
                        last_slc = int(i_slc-i_slc%10)
                        flaws = True
                        break

            elif i_slc in [10,20,30,40,50]:
                
                # Check if tenth slice of a ten chunk contains a flat field
                if(glob.glob(current_path + flat_name[1])):
                    # Update the last feasible tenth package in this scan_folder.
                    last_slc = i_slc
                else:
                    # Update the last feasible tenth package in this scan_folder.
                    last_slc = int(i_slc-10) # -10 because it fails at the tenth of tenth pack.
                    flaws = True
                    break

                # Check if all slices contain 3601 projections.
                for idx in projs_idx:
                    if(glob.glob(current_path + projs_name.format(projs_idx[idx]))):
                        # Update the last feasible tenth package in this scan_folder.
                        last_slc = i_slc
                    else:
                        # Update the last feasible tenth package in this scan_folder.
                        last_slc = int(i_slc-10) # -10 because it fails at the tenth of tenth pack.
                        flaws = True
                        break

            else:
                # Check if all slices contain 3601 projections.
                for idx in projs_idx:
                    if(glob.glob(current_path + projs_name.format(projs_idx[idx]))):
                        # Update the last feasible tenth package in this scan_folder.
                        last_slc = i_slc
                    else:
                        # Update the last feasible tenth package in this scan_folder.
                        last_slc = int(i_slc-i_slc%10)
                        flaws = True
                        break
            
            if flaws: break
        if flaws: break
            
    print("By sanity check 2:", folder," has", last_slc, " feasible slices.")
    folder_name = folder.rsplit('/')[-1]
    # Dictionary structure:
    # key = foldername
    # [0] = last feasible slice
    # [1] = original path
    good_slcs[folder_name]=[last_slc, folder]

print(np.round_(time.time() - t, 3), 'sec elapsed for checking all folders in', base_dir_str_1, 'and', base_dir_str_2, 'and', base_dir_str_add)

Sanity check the data


  0%|          | 0/140 [00:00<?, ?it/s]

By sanity check 1: /export/scratch1/mbk/2DeteCT_dataset/20220317_2308_50slices  has 50  feasible slices.
By sanity check 2: /export/scratch1/mbk/2DeteCT_dataset/20220317_2308_50slices  has 50  feasible slices.
By sanity check 1: /export/scratch1/mbk/2DeteCT_dataset/20220318_1314_40slices  has 40  feasible slices.
By sanity check 2: /export/scratch1/mbk/2DeteCT_dataset/20220318_1314_40slices  has 40  feasible slices.
By sanity check 1: /export/scratch1/mbk/2DeteCT_dataset/20220318_2220_50slices  has 50  feasible slices.
By sanity check 2: /export/scratch1/mbk/2DeteCT_dataset/20220318_2220_50slices  has 50  feasible slices.
By sanity check 1: /export/scratch1/mbk/2DeteCT_dataset/20220319_1227_40slices_err20  has 20  feasible slices.
By sanity check 2: /export/scratch1/mbk/2DeteCT_dataset/20220319_1227_40slices_err20  has 20  feasible slices.
By sanity check 1: /export/scratch1/mbk/2DeteCT_dataset/20220319_2318_50slices  has 50  feasible slices.
By sanity check 2: /export/scratch1/mbk/2De

By sanity check 1: /export/scratch1/mbk/2DeteCT_dataset/20220411_1306_50slices  has 50  feasible slices.
By sanity check 2: /export/scratch1/mbk/2DeteCT_dataset/20220411_1306_50slices  has 50  feasible slices.
By sanity check 1: /export/scratch1/mbk/2DeteCT_dataset/20220411_2345_50slices  has 50  feasible slices.
By sanity check 2: /export/scratch1/mbk/2DeteCT_dataset/20220411_2345_50slices  has 50  feasible slices.
By sanity check 1: /export/scratch1/mbk/2DeteCT_dataset/20220412_1152_50slices  has 50  feasible slices.
By sanity check 2: /export/scratch1/mbk/2DeteCT_dataset/20220412_1152_50slices  has 50  feasible slices.
By sanity check 1: /export/scratch1/mbk/2DeteCT_dataset/20220412_2212_50slices  has 50  feasible slices.
By sanity check 2: /export/scratch1/mbk/2DeteCT_dataset/20220412_2212_50slices  has 50  feasible slices.
By sanity check 1: /export/scratch1/mbk/2DeteCT_dataset/20220413_1036_50slices_err40  has 40  feasible slices.
By sanity check 2: /export/scratch1/mbk/2DeteCT_d

By sanity check 1: /export/scratch1/mbk/2DeteCT_dataset/20220611_0938_50slices  has 50  feasible slices.
By sanity check 2: /export/scratch1/mbk/2DeteCT_dataset/20220611_0938_50slices  has 50  feasible slices.
By sanity check 1: /export/scratch1/mbk/2DeteCT_dataset/20220611_2104_50slices  has 50  feasible slices.
By sanity check 2: /export/scratch1/mbk/2DeteCT_dataset/20220611_2104_50slices  has 50  feasible slices.
By sanity check 1: /export/scratch1/mbk/2DeteCT_dataset/20220612_1004_50slices  has 50  feasible slices.
By sanity check 2: /export/scratch1/mbk/2DeteCT_dataset/20220612_1004_50slices  has 50  feasible slices.
By sanity check 1: /export/scratch1/mbk/2DeteCT_dataset/20220612_2057_50slices  has 50  feasible slices.
By sanity check 2: /export/scratch1/mbk/2DeteCT_dataset/20220612_2057_50slices  has 50  feasible slices.
By sanity check 1: /export/scratch1/mbk/2DeteCT_dataset/20220613_1120_50slices  has 50  feasible slices.
By sanity check 2: /export/scratch1/mbk/2DeteCT_dataset

By sanity check 1: /export/scratch2/mbk/2DeteCT_dataset/20220716_1053_50slices  has 50  feasible slices.
By sanity check 2: /export/scratch2/mbk/2DeteCT_dataset/20220716_1053_50slices  has 50  feasible slices.
By sanity check 1: /export/scratch2/mbk/2DeteCT_dataset/20220716_2206_50slices  has 50  feasible slices.
By sanity check 2: /export/scratch2/mbk/2DeteCT_dataset/20220716_2206_50slices  has 50  feasible slices.
By sanity check 1: /export/scratch2/mbk/2DeteCT_dataset/20220717_1230_50slices_err40  has 40  feasible slices.
By sanity check 2: /export/scratch2/mbk/2DeteCT_dataset/20220717_1230_50slices_err40  has 40  feasible slices.
By sanity check 1: /export/scratch2/mbk/2DeteCT_add/20220309_1209_50slices_figs  has 50  feasible slices.
By sanity check 2: /export/scratch2/mbk/2DeteCT_add/20220309_1209_50slices_figs  has 50  feasible slices.
By sanity check 1: /export/scratch2/mbk/2DeteCT_add/20220310_0943_50slices_almonds  has 50  feasible slices.
By sanity check 2: /export/scratch2/m

In [4]:
# Create the folder structure for the dataset.
# Determine the total number of feasible slice n_slc ~ 5500.

n_slc = 0
for key in good_slcs:
    n_slc += good_slcs[key][0]

print("The total number of feasible slices is",n_slc)

# Define the numbers of modes to be processed.
n_modes = 3

save_path_local = "/export/scratch2/mbk/2DeteCT_tests/test1/" 

try:
    os.mkdir(save_path_local)
except OSError:
    print ("Creation of the directory %s failed" % save_path_local)
    if os.path.exists(save_path_local): print("The above path already exists.")
    else: print ("Successfully created the directory %s" % save_path_local)

for i_slc in range(n_slc):
    folder_name = (save_path_local + 'slice' + str(i_slc+1).zfill(5))
    os.mkdir(folder_name)
    for i_mode in range(n_modes):
        folder_name = (save_path_local + 'slice' + str(i_slc+1).zfill(5) + '/' 'mode' + str(i_mode+1))
        os.mkdir(folder_name)



The total number of feasible slices is 6370


In [5]:
# Keep track of the processing time per folder.
t = time.time();
print('load and process data into sinograms', flush=True)

# Create a dictionary to bookkeep the dataset structure.
dataset_2DeteCT = dict()

# Dictionary structure:
# key = slice number in the final dataset
# [0] = foldername
# [1] = original path


# Create slice counter for the keys of the dictionary.
slc_counter = 1

# Create sinograms, check size and copy all flat fields and dark fields to their appropiate location.
# Iterate through the scan_folders via the keys of the dictionary good_slcs to keep the order.

for key in tqdm_notebook(good_slcs.keys(), desc = 'Loop over all scan folders'):
    # Iterate through all feasible slices within this scan_folder.
    # [0] representing the last feasible slice.
    for i_slc in tqdm_notebook(np.arange(1,good_slcs[key][0]+1), desc = 'Loop over all feasible slices in each scan folder'):
        # +1 because it shall include the last.
        for i_mode in modes:
            current_path = good_slcs[key][1] + "/" + slcs_name.format(i_slc) + "/mode{}/".format(i_mode)
            
            # Copying the dark and flat fields if existent into all folders.
            
            if i_slc in [1,11,21,31,41]:
                # Copy file into all other folders of this ten chunk within the final dataset.
                for i_dest in np.arange(slc_counter, slc_counter+10):
                    dest_path = save_path_local + slcs_name.format(i_dest) + "/mode{}/".format(i_mode)
                    # Copy the pre flat field.
                    shutil.copy2(current_path + flat_name[0], dest_path + "flat1.tif") # Target filename is "flat1.tif".
                    # Copy the dark field.
                    shutil.copy2(current_path + dark_name, dest_path + "dark.tif") # Target filename is "dark.tif".
                    
                # Create the sinogram and save it into destination path.
                dest_path = save_path_local + slcs_name.format(slc_counter) + "/mode{}/".format(i_mode)
                projectionFiles = sorted(glob.glob(current_path + "scan_*.tif"))
                nPro = len(projectionFiles)
                # 1912 is the length of the 1D line projections.
                projs = np.zeros((nPro,1912), dtype='uint16') # 'uint16' is the dtype of the projection files.
                # Therefore in the following loop there is no type-conversions.
                for i in range(nPro):
                    projs[i] = imageio.imread(projectionFiles[i])
                sinogram = np.ascontiguousarray(projs)
                assert sinogram.shape == (3601,1912), \
                f"expected sinogram with shape (3601,1912), got: {sinogram.shape}"
                imageio.imwrite(str(dest_path + "sinogram.tif"),sinogram)
                    
            
            elif i_slc in [10,20,30,40,50]:
                # Copy file into all other folders of this ten chunk within the final dataset.
                for i_dest in np.arange(slc_counter-9,slc_counter+1):
                    dest_path = save_path_local + slcs_name.format(i_dest) + "/mode{}/".format(i_mode)
                    # Copy the post flat field.
                    shutil.copy2(current_path + flat_name[1], dest_path + "flat2.tif") # Target filename is "flat2.tif".
                
                # Create the sinogram and save it into destination path.
                dest_path = save_path_local + slcs_name.format(slc_counter) + "/mode{}/".format(i_mode)
                projectionFiles = sorted(glob.glob(current_path + "scan_*.tif"))
                nPro = len(projectionFiles) 
                # 1912 is the length of the 1D line projections.
                projs = np.zeros((nPro,1912), dtype='uint16') # 'uint16' is the dtype of the projection files.
                # Therefore in the following loop there is no type-conversions.
                for i in range(nPro):
                    projs[i] = imageio.imread(projectionFiles[i])
                sinogram = np.ascontiguousarray(projs)
                assert sinogram.shape == (3601,1912), \
                f"expected sinogram with shape (3601,1912), got: {sinogram.shape}"
                imageio.imwrite(str(dest_path + "sinogram.tif"),sinogram)
                
            else:
                dest_path = save_path_local + slcs_name.format(slc_counter) + "/mode{}/".format(i_mode)
                # Create the sinogram and save it into destination path.
                projectionFiles = sorted(glob.glob(current_path + "scan_*.tif"))
                nPro = len(projectionFiles)
                # 1912 is the length of the 1D line projections.
                projs = np.zeros((nPro,1912), dtype='uint16') # 'uint16' is the dtype of the projection files.
                # Therefore in the following loop there is no type-conversions.
                for i in range(nPro):
                    projs[i] = imageio.imread(projectionFiles[i])
                sinogram = np.ascontiguousarray(projs)
                assert sinogram.shape == (3601,1912), \
                f"expected sinogram with shape (3601,1912), got: {sinogram.shape}"
                imageio.imwrite(str(dest_path + "sinogram.tif"),sinogram)
            
        dataset_2DeteCT['slice' + str(slc_counter).zfill(5)]=[key, good_slcs[key][1] + "/" + slcs_name.format(i_slc)]
        slc_counter+=1

print(np.round_(time.time() - t, 3), 'sec elapsed for processing all scan folders in', base_dir_str_1,
      'and', base_dir_str_2, 'and', base_dir_str_add)

# Save the dictionary to disk.
np.save(save_path_local + 'dataset_2DeteCT.npy', dataset_2DeteCT)

load and process data into sinograms


Loop over all scan folders:   0%|          | 0/140 [00:00<?, ?it/s]

Loop over all feasible slices in each scan folder:   0%|          | 0/50 [00:00<?, ?it/s]

Loop over all feasible slices in each scan folder:   0%|          | 0/40 [00:00<?, ?it/s]

Loop over all feasible slices in each scan folder:   0%|          | 0/50 [00:00<?, ?it/s]

...

Loop over all feasible slices in each scan folder:   0%|          | 0/50 [00:00<?, ?it/s]

Loop over all feasible slices in each scan folder:   0%|          | 0/50 [00:00<?, ?it/s]

Loop over all feasible slices in each scan folder:   0%|          | 0/50 [00:00<?, ?it/s]

Loop over all feasible slices in each scan folder:   0%|          | 0/40 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
print(dataset_2DeteCT)

{'slice00001': ['20220317_2308_50slices', '/export/scratch1/mbk/2DeteCT_dataset/20220317_2308_50slices/slice00001'], 'slice00002': ['20220317_2308_50slices', '/export/scratch1/mbk/2DeteCT_dataset/20220317_2308_50slices/slice00002'], 'slice00003': ['20220317_2308_50slices', '/export/scratch1/mbk/2DeteCT_dataset/20220317_2308_50slices/slice00003'], 'slice00004': ['20220317_2308_50slices', '/export/scratch1/mbk/2DeteCT_dataset/20220317_2308_50slices/slice00004'], 'slice00005': ['20220317_2308_50slices', '/export/scratch1/mbk/2DeteCT_dataset/20220317_2308_50slices/slice00005'], 'slice00006': ['20220317_2308_50slices', '/export/scratch1/mbk/2DeteCT_dataset/20220317_2308_50slices/slice00006'], 'slice00007': ['20220317_2308_50slices', '/export/scratch1/mbk/2DeteCT_dataset/20220317_2308_50slices/slice00007'], 'slice00008': ['20220317_2308_50slices', '/export/scratch1/mbk/2DeteCT_dataset/20220317_2308_50slices/slice00008'], 'slice00009': ['20220317_2308_50slices', '/export/scratch1/mbk/2DeteCT_